In [15]:
import numpy as np
import random as r


def relu(array):
    return np.maximum(0, array)


class Layer_Dense:
    def __init__(self, d_inputs, n_neurons, n_inputs):
        self.d_inputs = d_inputs
        self.n_neurons = n_neurons
        self.n_inputs = n_inputs
        self.weights = np.empty((n_neurons, d_inputs))
        self.biases = np.empty((n_neurons, n_inputs))

    def forward(self, inputs):
        self.output= np.dot(self.weights, inputs) + self.biases

    def attrib_r(self):
      self.weights = 0.10 * np.random.randn(self.n_neurons, self.d_inputs)
      self.biases = np.tile(0.1 * np.random.randn(1, self.n_neurons).reshape(-1, 1), (1, self.n_inputs))

    def mut(self, p, cr):

        Lay = Layer_Dense(self.d_inputs, self.n_neurons, self.n_inputs)

        Lay.weights = self.weights.copy()
        Lay.biases = self.biases.copy()

        n_weights = self.n_neurons*self.d_inputs
        num_w_mod = int(n_weights * p)

        if num_w_mod > 0:
            indices = np.unravel_index(np.random.choice(n_weights, num_w_mod, replace=False), (self.n_neurons, self.d_inputs))
            changes_w = np.random.uniform(-cr, cr, num_w_mod)
            Lay.weights[indices] += changes_w

        n_b = self.n_neurons
        num_b_mod = int(n_b*p)+ (1 if r.random() < 0.5 else 0)

        if num_b_mod > 0:
            indices_b = np.unravel_index(np.random.choice(n_b, num_b_mod, replace=False), (n_b, 1))
            changes_b = np.random.uniform(-cr, cr, num_b_mod)
            Lay.biases[indices_b] += changes_b
        
        return(Lay)




layer1 = Layer_Dense(9, 5, 3)
layer1.attrib_r()
print(layer1.weights[0])
l = layer1.mut(0.5, 0.2)
print(l.weights[0])

[-0.00730597  0.02218433 -0.05788027 -0.05611244  0.13250169 -0.00626973
 -0.00134197  0.08059746 -0.08920854]
[-0.00730597 -0.01782344  0.12392741 -0.05611244  0.07064276  0.15093119
 -0.00134197  0.1233385  -0.08920854]


In [5]:
#Application voitures
import math as m
import pygame
import numpy as np
import sys
from pygame.constants import K_a
import random as r


pygame.init()
from pygame.constants import K_0, K_SPACE, K_a, K_DOWN, K_RIGHT, K_z

taille_fenetre = (1450, 800)
fenetre_width = 1450
fond = pygame.image.load(r"C:\Users\paulm\Desktop\Images\fond_voitures.jpg")
map_ugo = pygame.image.load(r"C:\Users\paulm\Downloads\Circuit.jpg")
circuit = pygame.image.load(r"C:\Users\paulm\Desktop\Images\Map_du_futur.jpg")
zigzag = pygame.image.load(r"C:\Users\paulm\Desktop\Images\map_zigzag.jpg")
fenetre = pygame.display.set_mode(taille_fenetre)
generation_font = pygame.font.SysFont("Arial", 70)
font = pygame.font.SysFont("Arial", 30)
spawn_point = (730, 666)
spawn_map = (0,0)
map = circuit
#Variables/Constantes
d = 22.36
a = 0.463
t =0
road = (54, 54, 54)
green = (61, 127, 1)
vert_accel = (0, 250, 0)
rouge_brake = (250, 0, 0)
grey = (100, 120, 130)
vert_ugo = (181, 230, 29)
elite_color = (200, 0, 200)

#Classes
class car:
    def __init__(self, x, y, e, o, c, dist, afk, name):
        self.x = x
        self.y = y
        self.e = e
        self.o = o
        self.c = c
        self.dist = dist
        self.afk = afk
        self.name = name
    def rect(self):
        x, y, beta, o = self.x, self.y, self.o - a, self.o
        x_a, y_a, c, s = x-(d*m.sin(beta)), y-(d*m.cos(beta)), m.cos(o), m.sin(o)
        return ((round(x_a),round(y_a)),(round(x_a + 40*s),round(y_a + 40*c)),(round(x_a -20*c+40*s),round(y_a +20*s+40*c)),(round(x_a -20*c), round(y_a +20*s)))
    def plot(self):
        pygame.draw.polygon(fenetre, self.c, self.rect())
        text = font.render(str(self.name), True, (255, 0, 255))
        text_rect = text.get_rect()
        text_rect.center = (self.x, self.y)
        fenetre.blit(text, text_rect)

    def move(self):
        o, e = self.o, self.e
        self.x += -m.sin(o)*e
        self.y += -m.cos(o)*e
        self.dist += self.e

    def collision(self):
        for cos in self.rect():
            if fenetre.get_at(cos) == green:
                return True
        return False
    def radar(self):
        state = [0]*5
        o, x, y = self.o, self.x, self.y
        for j in range(5):
            i = 0
            if fenetre.get_at((round(x), round(y))) != road:
                state[j] = 0
            else:
                while fenetre.get_at((round(x -i*m.sin(-1 + 0.5*j+ o)), round(y - i*m.cos(-1+ 0.5*j+o)))) == road and i< 140:
                    i+=12
                if fenetre.get_at((round(x -i*m.sin(-1 + 0.5*j+ o)), round(y - i*m.cos(-1+ 0.5*j+o)))) != road:
                    i-=1
                    while fenetre.get_at((round(x -i*m.sin(-1 + 0.5*j+ o)), round(y - i*m.cos(-1+ 0.5*j+o)))) != road:
                        i-=1
                    state[j] = (i+2)/140
                else:
                    state[j] = 1
        return state
    
    def radar_speed(self):
        r = self.radar()
        r.append(self.e)
        return r
    
    def plot_radar(self):
        state, x, y, o = self.radar(), self.x, self.y, self.o
        for j in range(5):
            pygame.draw.circle(fenetre, (250, 0, 0), (round(x -140*state[j]*m.sin(-0.8 + 0.4*j+ o)), round(y - 140*state[j]*m.cos(-0.8 + 0.4*j+o))), 2)

    def get_reward(self):
        return (self.dist / 1000)*(self.e/2)
    

def relu(array):
    return np.maximum(0, array)


class Layer_Dense:
    def __init__(self, d_inputs, n_neurons, n_inputs):
        self.d_inputs = d_inputs
        self.n_neurons = n_neurons
        self.n_inputs = n_inputs
        self.weights = np.empty((n_neurons, d_inputs))
        self.biases = np.empty((n_neurons, n_inputs))

    def forward(self, inputs):
        self.output= np.dot(self.weights, inputs) + self.biases

    def attrib_r(self):
      self.weights = 0.10 * np.random.randn(self.n_neurons, self.d_inputs)
      self.biases = np.tile(0.1 * np.random.randn(1, self.n_neurons).reshape(-1, 1), (1, self.n_inputs))

    def mut(self, p, cr):

        Lay = Layer_Dense(self.d_inputs, self.n_neurons, self.n_inputs)

        Lay.weights = self.weights.copy()
        Lay.biases = self.biases.copy()

        n_weights = self.n_neurons*self.d_inputs
        num_w_mod = int(n_weights * p)

        if num_w_mod > 0:
            indices = np.unravel_index(np.random.choice(n_weights, num_w_mod, replace=False), (self.n_neurons, self.d_inputs))
            changes_w = np.random.uniform(-cr, cr, num_w_mod)
            Lay.weights[indices] += changes_w

        n_b = self.n_neurons
        num_b_mod = int(n_b*p)+ (1 if r.random() < 0.5 else 0)

        if num_b_mod > 0:
            indices_b = np.unravel_index(np.random.choice(n_b, num_b_mod, replace=False), (n_b, 1))
            changes_b = np.random.uniform(-cr, cr, num_b_mod)
            Lay.biases[indices_b] += changes_b
        
        return(Lay)

class NeuralN:
    def  __init__(self, l):
        self.l = l

    def process(self, input):
        output = input
        for layers in self.l:
            layers.forward(output)
            output = layers.output
        return output.T

    def mutNN(self, p, cr):
        return NeuralN([layers.mut(p, cr) for layers in self.l])
    
    def attribNN_r(self):
        for layers in self.l:
            layers.attrib_r()

#Config IA
generation = 0
pop = 40
nb_apprentis_init = 28
p = 0.1 #Proportion des settings qui muttent dans un génome qui mute
cr = 0.35 #Range de mutation des coeffs qui mutent

while True:
    generation += 1
    #Récupérer l'élite
    if generation > 1:
        i_elite = fitness.index(max(fitness))
        best_fit = fitness[i_elite]
        #print("Fitness du meilleur: "+ str(best_fit))
        elite = NNs[i_elite]
        NNs = []
        NNs.append(elite)

        #Apprentissage dynamique
        coef_fit = (m.exp((-best_fit/5000)+1)/(1+m.exp((-best_fit/5000)+1)))
        n_app = nb_apprentis_init + int(m.log(best_fit, 2)) - 8

        for i in range(1, n_app+1):
            NNs.append(elite.mutNN(p, cr*coef_fit))
        for i in range(n_app+1, pop):
            NN = NeuralN([Layer_Dense(6, 4, 1)])
            NN.attribNN_r()
            NNs.append(NN)
    else:
        best_fit = 0
        NNs = []
        for i in range(pop):
            NN = NeuralN([Layer_Dense(6, 4, 1)])
            NN.attribNN_r()
            NNs.append(NN)
    
    alive = [True]*pop
    cars = []
    fitness = [0]*pop
    t, remain_cars, game_on = 0, pop, True

    for i in range(pop):
        Car = car(spawn_point[0], spawn_point[1], 1, 1.55, grey, 0, 0, "")
        cars.append(Car)
    while game_on:
        t+=1
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                sys.exit(0)

        fenetre.blit(map, spawn_map)

        #Neural Network
        for i, Car in enumerate(cars):
            if alive[i]:
                result = NNs[i].process(np.array([Car.radar_speed()]).T)
                choice = np.argmax(result[0])

                if result[0][choice] < 0:
                    choice = -1
                
                if choice == 0:
                    Car.e += 0.2/(((10*Car.e)+1))+0.02
                if choice == 1:
                    Car.e += -0.02 -0.03*Car.e
                    if Car.e < 0:
                        Car.e = 0
                if choice == 2:
                    Car.o -= 0.025
                if choice == 3:
                    Car.o += 0.025
                if Car.e > 0:
                    Car.e -= 0.003
                if Car.e < 0.04:
                    Car.afk += 1
                else:
                    Car.afk = 0
                Car.c = vert_accel if choice == 0 else(rouge_brake if choice == 1 else grey)
                if i == 0 and generation > 1:
                    Car.c = elite_color
                Car.move()
        
                if not Car.collision() and Car.afk < 6:
                    fitness[i] += Car.get_reward()
                else:
                    alive[i] = False
                    remain_cars -=1
        
        if remain_cars == 0:
            game_on = False

        for i, Car in enumerate(cars):
            if alive[i]:
                Car.plot()
        if alive[0]:
            cars[0].plot()

        if t > 3000:
            for i, Car in enumerate(cars):
                alive[i] = False
                game_on = False

        #Affichage
        text = generation_font.render("Generation : " + str(generation), True, (255, 255, 0))
        text_rect = text.get_rect()
        text_rect.center = (fenetre_width/2, 100)
        fenetre.blit(text, text_rect)

        text = font.render("Nb cars, best fitness: " + str(remain_cars)+", "+str(round(best_fit, 1)), True, (0, 50, 255))
        text_rect = text.get_rect()
        text_rect.center = (1260, 40)
        fenetre.blit(text, text_rect)

        pygame.display.flip()

        if pygame.key.get_pressed()[K_a]:
            pygame.quit()
            i_elite = fitness.index(max(fitness))
            for layers in NNs[i_elite].l:
                print([w for w in layers.weights], [b for b in layers.biases])

[array([ 0.06005398,  0.12134055,  0.29804968, -0.09605263, -0.01818744,
       -0.18096869]), array([ 0.16150636, -0.04521051,  0.0692569 , -0.00982629, -0.02267085,
       -0.1215121 ]), array([-0.15115799, -0.0269149 , -0.09860671, -0.01081255, -0.10366432,
        0.11580107]), array([-0.02913329, -0.01397202,  0.29051719,  0.01727647, -0.0069819 ,
       -0.03904338])] [array([0.10975381]), array([-0.04354829]), array([0.0626714]), array([-0.06288848])]


error: video system not initialized

Configuration d'une voiture pas mal:
Layer_Dense(5, 4, 1)
weights:
[[-0.11459844,  0.00366979,  0.14695304,  0.10900403,  0.19650179],[-0.0586525,  -0.15360717,  0.18208059, -0.17586387,  0.1774716 ],[-0.0492795,   0.1370558,  -0.08935747, -0.1381061,  -0.03477636],[ 0.02549292,  0.05906116,  0.15126756,  0.12251633, -0.11948816]]
biases:
[[ 0.04973856]
 [-0.1843661 ]
 [ 0.33566636]
 [ 0.00733744]]

[array([-0.14960911, -0.10875375, -0.00490328, -0.09209896,  0.09199594]), array([ 0.0411033 , -0.11216156, -0.01503237,  0.12966382,  0.00987674]), array([ 0.30936207, -0.21155632, -0.15254003, -0.10095604, -0.04125501]), array([ 0.10143836,  0.0277508 ,  0.07742792,  0.08973209, -0.18204887])] [array([0.28450594]), array([-0.02320083]), array([0.21727976]), array([-0.11538354])]

In [6]:
#Tester une voiture
#Application voitures
#Je clear tt neat, je fous juste un cerveau voir si elle conduit
import math as m
import pygame
import numpy as np
import sys
from pygame.constants import K_a
import random as r
import time

pygame.init()
from pygame.constants import K_0, K_SPACE, K_a, K_DOWN, K_RIGHT, K_z

taille_fenetre = (1450, 800)
fenetre_width = 1450
fond = pygame.image.load(r"C:\Users\paulm\Desktop\Images\fond_voitures.jpg")
map_ugo = pygame.image.load(r"C:\Users\paulm\Desktop\Images\Circuit.png")
circuit = pygame.image.load(r"C:\Users\paulm\Desktop\Images\Map_du_futur.jpg")
zigzag = pygame.image.load(r"C:\Users\paulm\Desktop\Images\map_zigzag.jpg")
fenetre = pygame.display.set_mode(taille_fenetre)
generation_font = pygame.font.SysFont("Arial", 70)
font = pygame.font.SysFont("Arial", 30)
spawn_point = (730, 666)
spawn_map = (0,0)
map = circuit
#Variables/Constantes
d = 22.36
a = 0.463
t =0
road = (54, 54, 54)
green = (61, 127, 1)
vert_accel = (0, 250, 0)
rouge_brake = (250, 0, 0)
grey = (100, 120, 130)
vert_ugo = (181, 230, 29, 255)
elite_color = (200, 0, 200)

#Classes
class car:
    def __init__(self, x, y, e, o, c, dist, afk, name):
        self.x = x
        self.y = y
        self.e = e
        self.o = o
        self.c = c
        self.dist = dist
        self.afk = afk
        self.name = name
    def rect(self):
        x, y, beta, o = self.x, self.y, self.o - a, self.o
        x_a, y_a, c, s = x-(d*m.sin(beta)), y-(d*m.cos(beta)), m.cos(o), m.sin(o)
        return ((round(x_a),round(y_a)),(round(x_a + 40*s),round(y_a + 40*c)),(round(x_a -20*c+40*s),round(y_a +20*s+40*c)),(round(x_a -20*c), round(y_a +20*s)))
    def plot(self):
        pygame.draw.polygon(fenetre, self.c, self.rect())
        text = font.render(str(self.name), True, (255, 0, 255))
        text_rect = text.get_rect()
        text_rect.center = (self.x, self.y)
        fenetre.blit(text, text_rect)

    def move(self):
        o, e = self.o, self.e
        self.x += -m.sin(o)*e
        self.y += -m.cos(o)*e
        self.dist += self.e

    def collision(self):
        for cos in self.rect():
            if fenetre.get_at(cos) == green:
                return True
        return False
    def radar(self):
        state = [0]*5
        o, x, y = self.o, self.x, self.y
        for j in range(5):
            i = 0
            if fenetre.get_at((round(x), round(y))) != road:
                state[j] = 0
            else:
                while fenetre.get_at((round(x -i*m.sin(-1 + 0.5*j+ o)), round(y - i*m.cos(-1+ 0.5*j+o)))) == road and i< 140:
                    i+=12
                if fenetre.get_at((round(x -i*m.sin(-1 + 0.5*j+ o)), round(y - i*m.cos(-1+ 0.5*j+o)))) != road:
                    i-=1
                    while fenetre.get_at((round(x -i*m.sin(-1 + 0.5*j+ o)), round(y - i*m.cos(-1+ 0.5*j+o)))) != road:
                        i-=1
                    state[j] = (i+2)/140
                else:
                    state[j] = 1
        return state
    
    def plot_radar(self):
        state, x, y, o = self.radar(), self.x, self.y, self.o
        for j in range(5):
            pygame.draw.circle(fenetre, (250, 0, 0), (round(x -140*state[j]*m.sin(-0.8 + 0.4*j+ o)), round(y - 140*state[j]*m.cos(-0.8 + 0.4*j+o))), 2)

    def get_reward(self):
        return (self.dist / 1000)*(self.e/2)
    

def relu(array):
    return np.maximum(0, array)


class Layer_Dense:
    def __init__(self, d_inputs, n_neurons, n_inputs):
        self.d_inputs = d_inputs
        self.n_neurons = n_neurons
        self.n_inputs = n_inputs
        self.weights = np.empty((n_neurons, d_inputs))
        self.biases = np.empty((n_neurons, n_inputs))

    def forward(self, inputs):
        self.output= np.dot(self.weights, inputs) + self.biases

    def attrib_r(self):
      self.weights = 0.10 * np.random.randn(self.n_neurons, self.d_inputs)
      self.biases = np.tile(0.1 * np.random.randn(1, self.n_neurons).reshape(-1, 1), (1, self.n_inputs))

    def mut(self, p, cr):

        Lay = Layer_Dense(self.d_inputs, self.n_neurons, self.n_inputs)

        Lay.weights = self.weights.copy()
        Lay.biases = self.biases.copy()

        n_weights = self.n_neurons*self.d_inputs
        num_w_mod = int(n_weights * p)

        if num_w_mod > 0:
            indices = np.unravel_index(np.random.choice(n_weights, num_w_mod, replace=False), (self.n_neurons, self.d_inputs))
            changes_w = np.random.uniform(-cr, cr, num_w_mod)
            Lay.weights[indices] += changes_w

        n_b = self.n_neurons
        num_b_mod = int(n_b*p)+ (1 if r.random() < 0.5 else 0)

        if num_b_mod > 0:
            indices_b = np.unravel_index(np.random.choice(n_b, num_b_mod, replace=False), (n_b, 1))
            changes_b = np.random.uniform(-cr, cr, num_b_mod)
            Lay.biases[indices_b] += changes_b
        
        return(Lay)

class NeuralN:
    def  __init__(self, l):
        self.l = l

    def process(self, input):
        output = input
        for layers in self.l:
            layers.forward(output)
            output = layers.output
        return output.T

    def mutNN(self, p, cr):
        return NeuralN([layers.mut(p, cr) for layers in self.l])
    
    def attribNN_r(self):
        for layers in self.l:
            layers.attrib_r()


NN = NeuralN([Layer_Dense(5, 4, 1)])
NN.l[0].weights = [[ 0.05997257,  0.08450915,  0.16671574,  0.11313827, -0.18264911], [ 0.03117921, -0.13529461,  0.08959896, -0.04010666, -0.37012114],[ 0.14988358, -0.14337408, -0.13942942, -0.48482629,  0.05267855],[-0.01399203, -0.21024397,  0.40545826, -0.1326028 ,  0.36091905]]
NN.l[0].biases =[[-0.16472454], [0.10312352], [0.356764], [-0.01072436]]

while True:
    Car = car(spawn_point[0], spawn_point[1], 1, 1.55, grey, 0, 0, "")
    game_on = True
    while game_on:
        t+=1
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                sys.exit(0)

        fenetre.blit(map, spawn_map)

        #Neural Network
        result = NN.process(np.array([Car.radar()]).T)
        choice = np.argmax(result[0])

        if result[0][choice] < 0:
            choice = -1
        
        if choice == 0:
            Car.e += 0.2/(((10*Car.e)+1))+0.02
        if choice == 1:
            Car.e += -0.02 -0.03*Car.e
            if Car.e < 0:
                Car.e = 0
        if choice == 2:
            Car.o -= 0.03
        if choice == 3:
            Car.o += 0.03
        if Car.e > 0:
            Car.e -= 0.003
        if Car.e < 0.04:
            Car.afk += 1
        else:
            Car.afk = 0
        Car.c = vert_accel if choice == 0 else(rouge_brake if choice == 1 else grey)
        Car.move()

        if not Car.collision() and Car.afk < 6:
            game_on = True
        else:
            game_on = False

        Car.plot()

        if t > 5000:
            game_on = False

        pygame.display.flip()

        if pygame.key.get_pressed()[K_a]:
            pygame.quit()

SystemExit: 0

C:\Users\paulm\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
